In [1]:
!pip install -q transformers datasets
!pip install -q peft
!pip install -q bitsandbytes
!pip install -q trl
!pip install -q accelerate
!pip install -q sentencepiece
!pip install -q langchain
!pip -q install chromadb
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/InhuKim/Chat-Web.git

Cloning into 'Chat-Web'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 55 (delta 11), reused 50 (delta 9), pack-reused 0
Receiving objects: 100% (55/55), 140.17 KiB | 1.32 MiB/s, done.
Resolving deltas: 100% (11/11), done.


# Import

In [20]:
from langchain.chains import LLMChain
import torch
import re
from tqdm import tqdm
import pandas as pd

In [4]:
import os
import sys
sys.path.append('/content/Chat-Web/RAG_Chatbot/')

In [5]:
from Agent import agent as ag

# prompt

- RAG 기반 챗봇을 위한 한국어 prompt
- txt 파일 형태를 읽어 langchain prompt 객체로 반환

In [6]:
prompt = ag.RAG_PROMPT('/content/Chat-Web/RAG_Chatbot/Agent/prompt_ko.txt').return_template()
print(prompt)

input_variables=['context', 'language', 'question', 'tag'] template='아래 context에 기반하여 question에 대답하세요:\n{context}\n\nQuestion: {question}\n\n뒤에 오는 주제에 기반하여 대답하세요: {tag}\n다음 언어에 맞춰서 answer에 대답을 출력하세요: {language}\n\nanswer :'


# NLU(Natural Language Understanding)

- pytorch로 huggingface에 올라온 오픈소스 모델을 활용하여 데이터셋에 맞춰 학습
- PLM(Pretrained Language Model)의 모델 config를 불러와서 학습 모델 로드
- 학습 모델은 드라이브에 업로드 필요(colab에서 업로드 시 정상적으로 업로드가 안되는 경우 있음)
- label dict은 모델이 분류 모델이기 때문에 다시 분류명으로 전환해 주는 작업 필요

In [7]:
nlu_config = {
    "model_name" : "klue/roberta-base",
    "max_seq_len" : 128,
    "out_features" : 10,
    "model_path" : "/content/drive/MyDrive/Colab_Notebooks/NLP/LLM/RoBERTa-base_AdamW_baseline_example.pth"
}

label_dict = {
    "0": "TS_가족과 주거",
    "1": "TS_경제 활동",
    "2": "TS_미용과 건강",
    "3": "TS_배움과 학문",
    "4": "TS_사회와 생활",
    "5": "TS_쇼핑과 상품",
    "6": "TS_스포츠",
    "7": "TS_식음료",
    "8": "TS_엔터테인먼트와 예술",
    "9": "TS_여가 활동"
}

In [8]:
nlu_llm = ag.NLU_CustomLLM(nlu_config, label_dict)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

# NLG(Natural Language Generation)

- langchain에서 활용하기 위한 custom LLM으로 로드
- huggingface에 업로드 된 모델명을 적으면 해당 모델을 다운로드 받아 적용
- 자동으로 4bit quantization 적용.

In [9]:
nlg_llm = ag.NLG_CustomLLM('davidkim205/komt-mistral-7b-v1')

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# Semantic Search

- 클론한 git repository의 DB 폴더의 load_data를 통해 csv파일의 문장을 DB에 임베딩 데이터 로드
- 데이터 벡터임베딩은 sentence-transformer의 paraphrase-multilingual-mpnet-base-v2 모델 활용
- DB 데이터는 구글드라이브에 업로드하여 활용

In [10]:
db = ag.RAG_DB(persist_path="/content/drive/MyDrive/Colab_Notebooks/NLP/knowledge_dataset/chroma_storage", collection_name="documents_collection")

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

# RAG Chatbot test

In [11]:
# 자연어생성 모델과 prompt template을 통해 llm chain 구현
answer_chain = LLMChain(llm=nlg_llm, prompt=prompt, output_key='answer')

In [ ]:
nlu_llm = ag.NLU_CustomLLM(nlu_config, label_dict)
nlg_llm = ag.NLG_CustomLLM('davidkim205/komt-mistral-7b-v1')
db = ag.RAG_DB(persist_path="/content/drive/MyDrive/Colab_Notebooks/NLP/knowledge_dataset/chroma_storage",
               collection_name="documents_collection")
answer_chain = LLMChain(llm=nlg_llm, prompt=prompt, output_key='answer')

In [12]:
# 질문 목록
input_text = "최초의 전자게임이 뭐야?"
input_text1 = "최초의 사진을 찍은 사람은 누구야?"
input_text2 = "이유식은 언제부터 주는게 좋을까요?"
input_text3 = "KF94 마스크의 자외선 차단 효과"

input_list = [input_text, input_text1, input_text2, input_text3]

#### DB 예시

In [13]:
doc = db.semantic_query(input_text)
doc

'처음으로 상업화한 전자 게임은 1972년에 미국 마그나북스사가 내놓은 1세대 비디오 게임기인 오디오 시스템이었어요. 트랜지스터, 콘덴서, 다이오드 등으로 만들어진 이 게임은 텔레비전 브라운관에 게임별 배경이 그려진 셀로판 종이를 부착하고 조종하는 사람이 선과 점을 움직이는 원초적인 수준이었죠. 하지만 광고에서는 탁구, 배구, 핸드볼, 축구, 골프, 장기, 사격 게임 등이 가능하다고 과장했어요. 1972년 가정용 비디오 게임 콘솔이 나오는데 마그나복스 오디세이라는 콘솔이야. 마그나복스 오디세이는 최초로 상용화된 콘솔인데 이 제품이 출시 된 시기를 1세대로 보고 시간이 흘러 현재 9세대까지 온 거야. 최초의 RPG는 앞서 살펴 보았던 던전 앤 드래곤이에요. 컴퓨터 RPG 게임의 시초는 울티마 시리즈로 유명한 리차드 개리엇이 1979년 발매한 아카라베스입니다. 울티마는 던전과 야외를 구분한 최초의 게임이고, 던전이 미로와 같이 구성되는 시스템을 도입하기도 했어요. 이러한 요소는 나중에 등장하는 일본식 RPG에도 영향을 주었죠.'

#### NLU 예시

In [14]:
# tag의 앞 3글자는 데이터셋 관련 메타정보. 제외 필요
tag = nlu_llm(input_text)
tag

'TS_스포츠'

#### 챗봇 질의응답 결과 예시

In [16]:
regex = r"answer : [a-zA-z0-9가-힣 ]+"
result = []

for text in tqdm(input_list):
    inputs_prompt = {
        "question" : text,
        "tag" : nlu_llm(text)[3:],
        "context" : db.semantic_query(text),
        "language" : 'korean'
    }
    response = answer_chain(inputs_prompt)
    match_list = re.search(regex, response['answer'], re.DOTALL)
    result.append(match_list.group(0))

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/t

In [18]:
display(*result)

'answer : 최초의 전자게임은 1972년에 미국 마그나북스사가 내놓은 1세대 비디오 게임기인 오디오 시스템이었습니다'

'answer : 조세프 니세포르 니에프스'

'answer : 대신 산딸기를 사 가시는 건 어때요'

'answer : KF94 마스크는 미세먼지 차단 효과가 뛰어나기 때문에 비말 차단용 마스크보다 더 효과적입니다'

In [19]:
doc_list = []
for text in tqdm(input_list):
    doc = db.semantic_query(text)
    doc_list.append(doc)

display(*doc_list)

100%|██████████| 4/4 [00:00<00:00, 23.88it/s]


'처음으로 상업화한 전자 게임은 1972년에 미국 마그나북스사가 내놓은 1세대 비디오 게임기인 오디오 시스템이었어요. 트랜지스터, 콘덴서, 다이오드 등으로 만들어진 이 게임은 텔레비전 브라운관에 게임별 배경이 그려진 셀로판 종이를 부착하고 조종하는 사람이 선과 점을 움직이는 원초적인 수준이었죠. 하지만 광고에서는 탁구, 배구, 핸드볼, 축구, 골프, 장기, 사격 게임 등이 가능하다고 과장했어요. 1972년 가정용 비디오 게임 콘솔이 나오는데 마그나복스 오디세이라는 콘솔이야. 마그나복스 오디세이는 최초로 상용화된 콘솔인데 이 제품이 출시 된 시기를 1세대로 보고 시간이 흘러 현재 9세대까지 온 거야. 최초의 RPG는 앞서 살펴 보았던 던전 앤 드래곤이에요. 컴퓨터 RPG 게임의 시초는 울티마 시리즈로 유명한 리차드 개리엇이 1979년 발매한 아카라베스입니다. 울티마는 던전과 야외를 구분한 최초의 게임이고, 던전이 미로와 같이 구성되는 시스템을 도입하기도 했어요. 이러한 요소는 나중에 등장하는 일본식 RPG에도 영향을 주었죠.'

'카메라 옵스큐라에 투영된 영상을 감광판으로 포착한 후 그 상을 물체에 정착시킨 최초의 사람은 조세프 니세포르 니에프스라는 프랑스인이야. 비투먼이라는 천연 아스팔트가 빛의 노출에 따라 굳는 성질을 이용해 1826년 그라의 창문에서 바라본 조망이라는 인류 최초의 사진이 니에프스에 의해 탄생했어. 사진술의 선구자였고, 세계 최초의 사진을 찍은 조세프 니세포르 니에프스는 프랑스의 발명가이자 사진가였어. 1880년대에 롤필름을 발명한 사람이 있어. 조지 이스트먼이라는 사람인데, 코닥을 설립했지. 당시 카메라를 사서 찍고, 현상소에 맡기면 끝이었어.'

'대신 산딸기를 사 가시는 건 어때요? 자연산 산딸기는 5월 말부터 6월 말까지가 제철이니까 지금이 적기거든요. 수분이 많은 과일은 가끔 간식으로 주는 것도 나쁘지 않아요. 대신 알러지 반응이 없는지 꼭 확인하시고, 과육만 작게 잘라서 주고, 씨는 급여하면 안돼요. 아이 키우기도 벅찬데 이유식까지 하려면 엄두가 안나긴 하겠지만 이유식에 너무 스트레스 받지마. 물론 이유식도 초기, 중기로 나뉘어져 권장되는 음식이 있는데 간단히 알아만 둬. 아이가 지금 6개월은 됐다고 했지? 미음부터 시작해서 8개월이 되면 5배 죽을 먹이면 돼. 그 후 야채와 고기, 과일 등을 먹여야 한다는 얘기가 있지만 꼭 지켜야 하는건 아니니 마음 편히 가져.'

'요즘 많이들 사용하는 KF94, KF80 마스크는 미세먼지 차단지수를 말하는 거지, 자외선을 차단하는 수치랑은 달라요. 마스크를 착용해도 선크림은 꼭 발라야 해요. 자외선 차단지수로 보면 KF94 마스크는 자외선 차단지수 SPF7정도예요. KF94처럼 천의 밀도가 높은 제품일수록 낮은 밀도의 마스크보다 자외선 차단 효과가 더 높아요. 천의 밀도가 낮은 제품은 숨쉬기는 편해서 좋은데, 자외선 투과율이 높은 단점이 있죠. 음, 미세먼지가 높을 때는 답답하더라도 비말 차단용 마스크 보단 KF94나 KF80처럼 미세 먼지 차단 효과가 뛰어난 마스크를 착용하는 게 좋지.'

In [22]:
df = pd.DataFrame({
    'question' : input_list[1:],
    'answer': result[1:]
})
df.to_csv('/content/drive/MyDrive/Colab_Notebooks/NLP/LLM/davidkim205_komt-mistral-7b-v1_answer.csv', index=0, encoding='utf-8-sig')

In [3]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/NLP/LLM/davidkim205_komt-mistral-7b-v1_answer.csv')
df

,question,answer
0,최초의 사진을 찍은 사람은 누구야?,answer : 조세프 니세포르 니에프스
1,이유식은 언제부터 주는게 좋을까요?,answer : 대신 산딸기를 사 가시는 건 어때요
2,KF94 마스크의 자외선 차단 효과,answer : KF94 마스크는 미세먼지 차단 효과가 뛰어나기 때문에 비말 차단용...
